# Cleaning Data

## Coordinates

In [22]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
import scipy as sc
import plotly.express as px

df = pd.read_csv('mcdonalds_dataset.csv')
df.head()

,lat,lon,alt,is_broken,is_active,dot,state,city,street,country,last_checked
0,-73.988281,40.718830,0,False,True,working,NY,New York,114 Delancey St,USA,Checked 142 minutes ago
1,-74.005090,40.728794,0,False,True,working,NY,New York,208 Varick St,USA,Checked 142 minutes ago
2,-73.993408,40.729197,0,False,True,working,NY,New York,724 Broadway,USA,Checked 142 minutes ago
3,-73.985855,40.726555,0,False,True,working,NY,New York,102 1st Ave,USA,Checked 142 minutes ago
4,-73.991692,40.691383,0,True,True,broken,NY,Brooklyn,82 Court St,USA,Checked 142 minutes ago


In [23]:
fig = px.density_mapbox(df, lat='lat', lon='lon', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.show()

There are no McDonalds in Antarctica but the dataset has latitudes pointing there. Obvious mistake, so lets swap the columns and rename them accordingly.

In [24]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

df = swap_columns(df, 'lat', 'lon')

df.rename(columns = {'lon':'latitude', 'lat':'longitude'}, inplace = True)
df.head()

,latitude,longitude,alt,is_broken,is_active,dot,state,city,street,country,last_checked
0,40.718830,-73.988281,0,False,True,working,NY,New York,114 Delancey St,USA,Checked 142 minutes ago
1,40.728794,-74.005090,0,False,True,working,NY,New York,208 Varick St,USA,Checked 142 minutes ago
2,40.729197,-73.993408,0,False,True,working,NY,New York,724 Broadway,USA,Checked 142 minutes ago
3,40.726555,-73.985855,0,False,True,working,NY,New York,102 1st Ave,USA,Checked 142 minutes ago
4,40.691383,-73.991692,0,True,True,broken,NY,Brooklyn,82 Court St,USA,Checked 142 minutes ago


In [25]:
fig = px.density_mapbox(df, lat='latitude', lon='longitude', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.show()

# https://plotly.com/python/mapbox-density-heatmaps/

#### References
swap_columns()
https://www.statology.org/swap-columns-pandas/

## Duplicates

In [26]:
#identify duplicate rows
duplicateRows = df[df.duplicated()]

#view duplicate rows
duplicateRows

# https://www.statology.org/pandas-find-duplicates/

,latitude,longitude,alt,is_broken,is_active,dot,state,city,street,country,last_checked


## Missing Data

In [27]:
df.isnull().sum().sum()

3954

In [28]:
missingdf = df.loc[pd.isnull(df).any(1),:]
fig = px.density_mapbox(missingdf, lat='latitude', lon='longitude', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.show()
missingdf

# https://stackoverflow.com/questions/36985505/keep-only-null-values-in-column-row-pandas

,latitude,longitude,alt,is_broken,is_active,dot,state,city,street,country,last_checked
12725,43.654620,-79.381000,0,False,True,working,NaN,Toronto,"(Food Court) Urban Eatery, 260 Yonge Street, U...",CA,Checked 11 minutes ago
12726,43.650890,-79.378450,0,False,True,working,NaN,Toronto,123 Yonge St,CA,Checked 11 minutes ago
12727,43.658210,-79.381900,0,False,True,working,NaN,Toronto,356 Yonge Street,CA,Checked 11 minutes ago
12728,43.653110,-79.375520,0,False,True,working,NaN,Toronto,127 Church Street,CA,Checked 11 minutes ago
12729,43.647080,-79.378570,0,False,True,working,NaN,Toronto,"(Food Court) 181 Bay St, P.O. Box 112",CA,Checked 11 minutes ago
...,...,...,...,...,...,...,...,...,...,...,...
16666,52.514265,13.475643,0,False,False,inactive,NaN,Berlin,Frankfurter Allee 117,DE,Checked 31 minutes ago
16667,54.076239,13.429812,0,False,False,inactive,NaN,Greifswald,Anklamer Landstr. 1,DE,Checked 31 minutes ago
16668,53.100934,8.787059,0,False,False,inactive,NaN,Bremen,Waller Heerstr. 101,DE,Checked 31 minutes ago
16669,53.628227,11.409059,0,False,False,inactive,NaN,Schwerin,Marienplatz 5-7,DE,Checked 31 minutes ago


In [29]:
df.state.isnull().sum()

3946

In [30]:
df.city.isnull().sum()

8

In [31]:
3954-3946

8

These are all the missing values. All the missing values in 'state' column are non USA - lets fill them in as 'non_usa'

In [32]:
# replacing na values in state with non_usa
df["state"].fillna("non_usa", inplace = True)
df.state.isnull().sum()

0

In [33]:
missingdf = df.loc[pd.isnull(df).any(1),:]

fig = px.density_mapbox(missingdf, lat='latitude', lon='longitude', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.show()
missingdf

,latitude,longitude,alt,is_broken,is_active,dot,state,city,street,country,last_checked
14137,51.505407,-0.084890,0,False,True,working,non_usa,NaN,Tooley Street,UK,Checked 17 minutes ago
14696,52.687450,-1.548397,0,False,True,working,non_usa,NaN,Atherstone Road,UK,Checked 16 minutes ago
14810,53.088390,-1.375880,0,False,True,working,non_usa,NaN,Hockley Way,UK,Checked 16 minutes ago
14943,53.744917,-0.341025,0,False,True,working,non_usa,NaN,39-41 Jameson Street,UK,Checked 15 minutes ago
15146,53.794990,-2.246980,0,False,True,working,non_usa,NaN,Princess Way,UK,Checked 15 minutes ago
15257,55.010379,-1.496732,0,False,True,working,non_usa,NaN,Silverlink Retail Park,UK,Checked 14 minutes ago
15287,54.599170,-5.930170,0,False,True,working,non_usa,NaN,2-4 Donegal Place,UK,Checked 14 minutes ago
15370,55.892067,-4.331650,0,False,True,working,non_usa,NaN,1841 Great Western Road,UK,Checked 14 minutes ago


In [34]:
list(missingdf.index.values)
# https://www.statology.org/pandas-index-to-list/

[14137, 14696, 14810, 14943, 15146, 15257, 15287, 15370]

As seen from the map above these index locations are: London, Derby, Derby, Hull, Burnley, Gateshead, Belfast, Glasgow

In [35]:
df.at[14137, 'city'] = 'London'
df.at[14696, 'city'] = 'Derby'
df.at[14810, 'city'] = 'Derby'
df.at[14943, 'city'] = 'Hull'
df.at[15146, 'city'] = 'Burnley'
df.at[15257, 'city'] = 'Gateshead'
df.at[15287, 'city'] = 'Belfast'
df.at[15370, 'city'] = 'Glasgow'
df.isnull().sum().sum()


0

## Structular Changes

In [36]:
# Remove text from 'last_checked'

df = df.replace(to_replace={'Checked ', ' minutes ago'}, value='', regex=True)
# https://datagy.io/pandas-replace-values/

df

,latitude,longitude,alt,is_broken,is_active,dot,state,city,street,country,last_checked
0,40.718830,-73.988281,0,False,True,working,NY,New York,114 Delancey St,USA,142
1,40.728794,-74.005090,0,False,True,working,NY,New York,208 Varick St,USA,142
2,40.729197,-73.993408,0,False,True,working,NY,New York,724 Broadway,USA,142
3,40.726555,-73.985855,0,False,True,working,NY,New York,102 1st Ave,USA,142
4,40.691383,-73.991692,0,True,True,broken,NY,Brooklyn,82 Court St,USA,142
...,...,...,...,...,...,...,...,...,...,...,...
16666,52.514265,13.475643,0,False,False,inactive,non_usa,Berlin,Frankfurter Allee 117,DE,31
16667,54.076239,13.429812,0,False,False,inactive,non_usa,Greifswald,Anklamer Landstr. 1,DE,31
16668,53.100934,8.787059,0,False,False,inactive,non_usa,Bremen,Waller Heerstr. 101,DE,31
16669,53.628227,11.409059,0,False,False,inactive,non_usa,Schwerin,Marienplatz 5-7,DE,31


In [37]:
# rename as 'last_checked_minutes_ago'
df.rename(columns = {'last_checked':'last_checked_minutes_ago'}, inplace = True)

In [38]:
df.last_checked_minutes_ago

0        142
1        142
2        142
3        142
4        142
        ... 
16666     31
16667     31
16668     31
16669     31
16670     31
Name: last_checked_minutes_ago, Length: 16671, dtype: object

In [39]:
pd.set_option('display.max_columns', 100)
df.last_checked_minutes_ago.head()

0    142
1    142
2    142
3    142
4    142
Name: last_checked_minutes_ago, dtype: object

In [40]:
df.last_checked_minutes_ago.unique()

array(['142', '143', '144', '145', '146', '147', '148', '149', '150',
       '151', '152', '153', '154', '155', '156', '157', '158', '159',
       '160', '161', '162', '163', '164', '165', '166', '167', '168',
       '169', '171', '170', '172', '173', '2', '174', '3', '4', '5', '6',
       '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '40',
       '39', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50',
       '51', '52', '54', '53', '55', '56', '57', '58', '60', '61', '63',
       '62', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73',
       '75', '74', '77', '76', '78', '79', '80', '81', '82', '83', '84',
       '86', '85', '87', '88', '89', '90', '91', '92', '93', '94', '95',
       '96', '97', '98', '100', '99', '101', '102', '103', '104', '105',
       '106', '107', '108', '109', '110', '111', '112', '113',

In [41]:
df = df.replace(to_replace={' minute ago'}, value='', regex=True)

In [42]:
df.last_checked_minutes_ago.astype(int)

0        142
1        142
2        142
3        142
4        142
        ... 
16666     31
16667     31
16668     31
16669     31
16670     31
Name: last_checked_minutes_ago, Length: 16671, dtype: int64